<a href="https://colab.research.google.com/github/Jerryson520/Mortgage-Backed-Securities-MBS-Cashflow-Waterfall-Model/blob/main/Cashflow_Waterfall_Model_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mortgage-Backed Securities Cashflow Waterfall Model
Implement a cashflow waterfall model for a hypothetical Mortgage-Backed Security using provided mortgage data.


Given:
- A file containing data for 15 mortgages ($100million total balance), including their respective unpaid principal balances, interest
rates (annual), and terms (in months).

Bond Structure Description:
- Three classes of bonds: A, B, and C
- Principal paid sequentially: Class A gets ALL aggregated principal from the collateral paid each period until its balance has reached 0.
Then, once Class A is paid oﬀ, Class B begins to get ALL principal each period until its balance has reached 0.
Then, once Class A and Class B are paid oﬀ, Class C is paid the ALL remaining principal each period until its balance has reached 0.
- Interest paid sequentially to all bonds in every period (A, then B, then C): Unlike principal payments, each bond will get its interest
payments in every period based on their then current balance.
- Initial principal balances:
  * Class A: $70,000,000

  * Class B: $20,000,000

  * Class C: $10,000,000
- Interest rates for all bonds: Equal to the gross weighted average interest rate of the collateral (weighted by then current collateral
balance)

Tasks:
1. Design and implement the waterfall mechanism to distribute the aggregated cashflows to the bonds according to the
specified structure:
* Payment Sequence:
  * Pay interest to all bonds sequentially (A, then B, then C) in each period from the period’s aggregate interest.
  * Pay principal to Class A bond from the period’s aggregate principal, until fully paid oﬀ.
  * Once Class A is paid oﬀ, pay principal to Class B bond from the period’s aggregate principal, until fully paid oﬀ.
  * Once Class B is paid oﬀ, pay principal to Class C bond from the period’s aggregate principal, until fully paid oﬀ.

In [ ]:
# !pip install numpy_financial
!pip install plotly

In [ ]:
import pandas as pd, numpy as np
# import numpy_financial as npf # use npf.irr function to calculate irr
import matplotlib.pyplot as plt, seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# DataLoader class

In [ ]:
class DataLoader:
    """
    A class to load data from various file formats.

    Attributes
    ----------
    file_path : str
        The path to the data file.
    headers : list of str
        The list of column headers to use for the data.

    Methods
    -------
    load_data():
        Determines the file type and loads the data accordingly.
    """

    def __init__(self, file_path):
        """
        Initializes the DataLoader with the specified file path.

        Parameters
        ----------
        file_path : str
            The path to the data file.
        """
        self.file_path = file_path
        self.headers = ['ID', 'Cut_Off_Date_Balance', 'Gross_Coupon', 'Remaining_Amortization'] # because of space in original headers


    def load_data(self):
        """
        Loads data from the specified file path based on its extension.

        Returns
        -------
        DataFrame
            A pandas DataFrame containing the loaded data.

        Raises
        ------
        ValueError
            If the file extension is not supported.
        """
        _, file_extension = os.path.splitext(self.file_path)

        if file_extension == '.xlsx':
            return self._load_excel()
        elif file_extension == '.csv':
            return self._load_csv()
        elif file_extension == '.npy':
            return self._load_npy()
        else:
            raise ValueError("Unsupported file type: {}".format(file_extension))


    def _load_excel(self):
        """
        Loads data from an Excel file.

        Returns
        -------
        DataFrame
            A pandas DataFrame containing the loaded data.
        """
        return pd.read_excel(self.file_path, names=self.headers)


    def _load_csv(self):
        """
        Loads data from a CSV file.

        Returns
        -------
        DataFrame
            A pandas DataFrame containing the loaded data.
        """
        return pd.read_csv(self.file_path, names=self.headers)


    def _load_npy(self):
        """
        Loads data from a NumPy file.

        Returns
        -------
        DataFrame
            A pandas DataFrame containing the loaded data.
        """
        data = np.load(self.file_path, allow_pickle=True)
        return pd.DataFrame(data, columns=self.headers)

# Mortgage Loan class

In [ ]:
class MortgageLoan:
    """
    A class to represent a mortgage loan.

    Attributes
    ----------
    loan_id : float
        Each mortgage loan's individual loan ID.
    initial_balance : float
        The initial balance of the mortgage loan.
    current_balance : float
        The current outstanding balance of the mortgage loan.
    gross_coupon : float
        The monthly interest rate derived from the annual gross coupon rate.
    amortization : int
        The total number of months from issuance to maturity.
    months_elapsed : int
        The number of months that have passed since the loan was issued.

    Methods
    -------
    monthly_payment()
        Calculates the fixed monthly payment required to amortize the loan over its term.
    monthly_interest()
        Calculates the interest for the current month based on the current balance.
    update_monthly()
        Updates the loan's balance and elapsed months after a monthly payment is made.
    distribute_cashflow()
        Generates a DataFrame representing the cash flow distribution over the loan's term.
    """

    def __init__(self, loan_id, initial_balance, gross_coupon, amortization):
        """
        Initializes a MortgageLoan instance with the specified attributes.

        Parameters
        ----------
        loan_id : float
            The unique identifier for the mortgage loan.
        initial_balance : float
            The starting balance of the mortgage loan.
        gross_coupon : float
            The annual interest rate of the mortgage loan (as a percentage).
        amortization : int
            The number of months over which the loan will be amortized.
        """
        self.loan_id = loan_id
        self.initial_balance, self.current_balance = initial_balance, initial_balance
        self.gross_coupon = gross_coupon / (12 * 100) # change annual to monthly (using compound method)
        self.amortization = amortization
        self.months_elapsed = 0

    def __repr__(self):
        """
        Returns a string representation of the MortgageLoan instance.

        Returns
        -------
        str
            A string representation of the mortgage loan, including its ID, initial balance,
            gross coupon rate, and remaining amortization period.
        """
        return (f"MortgageLoan(ID={self.loan_id}, Initial_Balance={self.initial_balance}, "
                f"Gross_Coupon={self.gross_coupon}, Remaining_Amortization={self.amortization})")

    def monthly_payment(self):
        """
        Calculates the fixed monthly payment for the mortgage loan.

        Returns
        -------
        float
            The amount of the monthly payment required to amortize the loan over its term.
        """
        # use the formula of geometric sequence
        # PMT = P * r * (1+r)^n / [(1+r)^n - 1]
        p, r, n = self.initial_balance, self.gross_coupon, self.amortization
        PMT = p * r * (1 + r)**n / ((1 + r)**n - 1)
        return PMT

    def monthly_interest(self):
        """
        Calculates the interest for the current month.

        Returns
        -------
        float
            The interest amount for the current month based on the current balance.
        """
        return (self.current_balance * self.gross_coupon)

    def update_monthly(self):
        """
        Updates the loan's balance and elapsed months after a monthly payment.

        Returns
        -------
        dict
            A dictionary containing the monthly payment, interest, principal reduction,
            and new balance after the payment.
        """
        monthly_pay = self.monthly_payment()
        monthly_interest = self.monthly_interest()
        principal_reduction = monthly_pay - monthly_interest

        self.current_balance -= principal_reduction
        self.months_elapsed += 1

        return {
            'payment': monthly_pay,
            'interest': monthly_interest,
            'principal': principal_reduction,
            'new_balance': self.current_balance
        }

    def distribute_cashflow(self):
        """
        Generates a DataFrame representing the cash flow distribution over the loan's term.

        Returns
        -------
        DataFrame
            A pandas DataFrame containing the starting balance, interest, principal,
            cashflow, and ending balance for each month of the loan's term.
        """
        dict_loan = {
            'starting_balance': [],
            'interest': [],
            'principal': [],
            'cashflow': [],
            'ending_balance': []
        }
        length = int(self.amortization)
        for _ in range(length):
            dict_loan['starting_balance'].append(self.current_balance)

            dict1 = self.update_monthly()
            dict_loan['interest'].append(dict1['interest'])
            dict_loan['principal'].append(dict1['principal'])
            dict_loan['cashflow'].append(dict1['payment'])
            dict_loan['ending_balance'].append(dict1['new_balance'])

        return pd.DataFrame(dict_loan)

# Mortgage Pool class

In [ ]:
class MortgagePool:
    """
    A class to represent a pool of mortgage loans.

    Attributes
    ----------
    mortgage_loans : list of MortgageLoan
        A list of MortgageLoan objects representing individual mortgage loans in the pool.

    Methods
    -------
    aggregate_cashflow():
        Calculates the cash flow for each individual mortgage loan and aggregates the results.
    """
    def __init__(self, mortgage_loans):
        """
        Initializes a MortgagePool instance with a list of mortgage loans.

        Parameters
        ----------
        mortgage_loans : list of MortgageLoan
            A list of MortgageLoan objects to be included in the pool.
        """
        self.mortgage_loans = mortgage_loans
        self.max_period = int(max([loan.amortization for loan in mortgage_loans]))

    def aggregate_cashflow(self):
        """
        Calculates the cash flow for each individual mortgage loan and aggregates the results.

        Returns
        -------
        tuple
            A tuple containing:
            - individual_mortgage_cashflows: list of DataFrame
                A list of DataFrames, each representing the cash flow of an individual mortgage loan.
            - aggregate_mortgage_cashflow: DataFrame
                A DataFrame containing the aggregated cash flow of the mortgage pool, including
                effective coupon rates for each period.
        """
        # calculate individual mortgage loan's cashflow
        individual_mortgage_cashflows = []
        for mortgage_loan in self.mortgage_loans:
            individual_mortgage_cashflows.append(
                mortgage_loan.distribute_cashflow().reindex(range(self.max_period)).fillna(0) # DataFrame format consistency when aggregate
            )

        # Effective coupon (annual)
        effective_coupon = []
        for i in range(self.max_period):
            balance_vec = np.array([df.iloc[i]['starting_balance'] for df in individual_mortgage_cashflows])
            effective_coupon.append(
                (balance_vec/np.sum(balance_vec)) @ np.array([loan.gross_coupon*12*100 for loan in self.mortgage_loans]) # convert monthly to annual
            )


        # Aggregate final result
        aggregate_mortgage_cashflow = pd.concat(
            [
                pd.DataFrame({'period':  list(range(1, self.max_period+1))}),
                sum(individual_mortgage_cashflows),
                pd.DataFrame({'effective_coupon': effective_coupon})
            ],
            axis=1
        )
        return individual_mortgage_cashflows, aggregate_mortgage_cashflow

# Bond class

In [ ]:
class Bond:
    """
    A class to represent a bond.

    Attributes
    ----------
    class_name : str
        The tranche of the bond class.
    initial_balance : float
        The initial balance of the bond.
    current_balance : float
        The current outstanding balance of the bond.

    Methods
    -------
    update_monthly(effective_coupon, principal=0)
        Updates the bond's balance based on the effective coupon rate and principal payment.
    """

    def __init__(self, class_name, initial_balance, current_balance):
        """
        Initializes a Bond instance with the specified attributes.

        Parameters
        ----------
        class_name : str
            The tranche of the bond class.
        initial_balance : float
            The starting balance of the bond.
        current_balance : float
            The current outstanding balance of the bond.
        """
        self.class_name = class_name
        self.initial_balance = initial_balance
        self.current_balance = current_balance

    def __repr__(self):
        """
        Returns a string representation of the Bond instance.

        Returns
        -------
        str
            A string representation of the bond, including its class name, initial balance,
            and current balance.
        """
        return (f"Class_name(Name={self.class_name}, Initial_Balance={self.initial_balance}, "
                f"Current_Balance={self.current_balance})")

    def update_monthly(self, effective_coupon, principal=0):
        """
        Updates the bond's balance based on the effective coupon rate and principal payment.

        Parameters
        ----------
        effective_coupon : float
            The annual effective coupon rate (as a percentage).
        principal : float, optional
            The principal amount to be paid (default is 0).

        Returns
        -------
        tuple
            A tuple containing:
            - current_balance: float
                The updated current balance of the bond.
            - interest: float
                The interest amount for the month.
            - principal: float
                The principal amount paid.
            - cash_flow: float
                The total cash flow for the month (interest + principal).
        """

        interest = self.current_balance * (effective_coupon/12/100)
        cash_flow = interest + principal
        self.current_balance -= principal

        return (self.current_balance, interest, principal, cash_flow)

# Waterfall class

In [ ]:
class Waterfall:
    """
    A class to represent the cash flow distribution process for a set of bonds.

    Attributes
    ----------
    aggregate_mortgage_cashflow : pd.DataFrame
        A DataFrame containing the start/end balance, cash flows (interest & principal), and effective coupon rate.
    bonds : list of Bond
        A list of Bond objects representing the bonds involved in the cash flow distribution.

    Methods
    -------
    distribute_cashflow()
        Distributes the cash flow from the aggregate to the bonds based on their current balances and coupon rates.
    animate_cashflow_diagram(bonds_cashflows)
        Creates an animated Sankey diagram to visualize the cash flow distribution over time, separating principal and interest flows.
    get_summary_stats(bonds_cashflows)
        Calculates summary statistics for the bonds, including Weighted Average Life (WAL), total cash flow, and internal rate of return (IRR).
    """

    def __init__(self, aggregate_mortgage_cashflow, bonds):
        """
        Initializes a Waterfall instance with the specified attributes.

        Parameters
        ----------
        aggregate_mortgage_cashflow : pd.DataFrame
            A DataFrame containing the start/end balance, cash flows (interest & principal), and effective coupon rate.
        bonds : list of Bond
            A list of Bond objects representing the bonds involved in the cash flow distribution.
        """
        self.aggregate_mortgage_cashflow = aggregate_mortgage_cashflow
        self.bonds = bonds
        self.max_period = len(aggregate_mortgage_cashflow)


    def distribute_cashflow(self):
        """
        Distributes the cash flow from the aggregate_mortgage_cashflow to the bonds.

        Iterates over each period in the aggregate_mortgage_cashflow DataFrame, distributing the principal and calculating
        interest for each bond based on the effective coupon rate. Updates each bond's balance and records
        the cash flow distribution.

        Returns
        -------
        pd.DataFrame
            A DataFrame containing the cash flow distribution and outstanding balances for each bond.
        """
        dict_bond = {
            bond.class_name: {
                (bond.class_name + i): []
                for i in ['_balance', '_interest', '_principal', '_cashflow']
            } for bond in self.bonds
        }


        for i, row in self.aggregate_mortgage_cashflow.iterrows():
            pp = row['principal']
            eff_coupon = row['effective_coupon']

            for bond in self.bonds: # skip when balance = 0
                if bond.current_balance <= 0:
                    continue

                p = min(bond.current_balance, pp)
                pp -= p
                a, b, c, d = bond.update_monthly(eff_coupon, p)

                dict_bond[bond.class_name][(bond.class_name+'_balance')].append(a)
                dict_bond[bond.class_name][(bond.class_name+'_interest')].append(b)
                dict_bond[bond.class_name][(bond.class_name+'_principal')].append(c)
                dict_bond[bond.class_name][(bond.class_name+'_cashflow')].append(d)


        bonds_cashflows = pd.concat(
            [
                pd.DataFrame({'period': list(range(1, self.max_period+1))}),
                pd.DataFrame(dict_bond['A']).reindex(range(self.max_period)).fillna(0),
                pd.DataFrame(dict_bond['B']).reindex(range(self.max_period)).fillna(0),
                pd.DataFrame(dict_bond['C']).reindex(range(self.max_period)).fillna(0)
            ],
            axis=1
        )

        return bonds_cashflows


    def animate_cashflow_diagram(self, bonds_cashflows):
        """
        Creates an animated Sankey diagram to visualize the cash flow distribution over time, separating principal and interest flows.

        Parameters
        ----------
        bonds_cashflows : pd.DataFrame
            A DataFrame containing cash flow data for bonds, including principal and interest payments, and balances.

        Returns
        -------
        None
        """
        # Prepare the data by dropping unnecessary columns and grouping by period
        allu = bonds_cashflows.drop(['A_balance', 'B_balance', 'C_balance'], axis=1)
        allu['period'] = (allu['period']-1)//12 + 1
        allu_year = allu.groupby('period').sum().reset_index()

        allu_year = pd.concat(
            [allu_year, bonds_cashflows.loc[list(range(11, 360, 12)), ['A_balance', 'B_balance', 'C_balance']].reset_index(drop=True)],
            axis=1
        )

        # Define colors for each link
        link_colors = [
            'rgba(39, 245, 241, 0.8)',  # Light Blue
            'rgba(255, 127, 14, 0.8)',   # Orange
            'rgba(44, 160, 44, 0.8)',    # Green
            'rgba(214, 39, 40, 0.8)',    # Red
            'rgba(148, 103, 189, 0.8)',  # Purple
            'rgba(227, 119, 194, 0.8)',  # Pink
            'rgba(31, 119, 180, 0.8)',   # Dark Blue
            'rgba(255, 180, 0, 0.8)',    # Yellow
            'rgba(127, 60, 141, 0.8)'    # Dark Purple
        ]


        # Define labels for the nodes in the principal and interest diagrams
        labels_principal = ['mortgage pool remaining balance', 'BondA remaining balance', 'BondB remaining balance', 'BondC remaining balance',
                           'mortgage pool principal', 'BondA principal payment', 'BondB principal payment', 'BondC principal payment']

        labels_interest = ['mortgage pool interest', 'BondA interest', 'BondB interest', 'BondC interest']

        # Create subplots for principal and interest flows
        fig = make_subplots(
            rows=2, cols=1,
            specs=[[{"type": "domain"}], [{"type": "domain"}]],  # Specify 'domain' type
            # vertical_spacing=0.6,
            # subplot_titles=("Principal Flows") # , "Interest Flows"
        )

        # Calculate portions for the initial state
        portion0 = sum([allu_year.iloc[0]['A_interest'], allu_year.iloc[0]['B_interest'], allu_year.iloc[0]['C_interest']]) / 100000000
        portion01 = (0.95 - portion0) * sum([allu_year.iloc[0]['A_balance'], allu_year.iloc[0]['B_balance'], allu_year.iloc[0]['C_balance'], allu_year.iloc[0]['A_principal'], allu_year.iloc[0]['B_principal'], allu_year.iloc[0]['C_principal']]) / 100000000

        # Add the initial state of the Sankey diagrams to the subplots
        fig.add_trace(
            go.Sankey(
                domain=dict(x=[0, 1], y=[0.95*portion0+0.05, 0.95*portion0+0.05+portion01]),
                node=dict(
                    pad=15,  # Increased padding value to separate nodes more clearly
                    thickness=20,
                    line=dict(color="black", width=0.5),
                    label=labels_principal,
                    color="black"
                ),
                link=dict(
                    source=[0, 0, 0, 4, 4, 4],
                    target=[1, 2, 3, 5, 6, 7],
                    value=[
                        allu_year.iloc[0]['A_balance'], allu_year.iloc[0]['B_balance'], allu_year.iloc[0]['C_balance'],
                        allu_year.iloc[0]['A_principal'], allu_year.iloc[0]['B_principal'], allu_year.iloc[0]['C_principal']
                    ],
                    color=link_colors[:6]
                )
            ), row=1, col=1
        )

        fig.add_trace(
            go.Sankey(
                domain=dict(x=[0, 1], y=[0, 0.95*portion0]),
                node=dict(
                    pad=15,  # Increased padding value to separate nodes more clearly
                    thickness=20,
                    line=dict(color="black", width=0.5),
                    label=labels_interest,
                    color="black"
                ),
                link=dict(
                    source=[0, 0, 0],
                    target=[1, 2, 3],
                    value=[
                        allu_year.iloc[0]['A_interest'], allu_year.iloc[0]['B_interest'], allu_year.iloc[0]['C_interest']
                    ],
                    color=link_colors[6:]
                )
            ), row=2, col=1
        )


        # Create frames for each period to synchronize both Sankey diagrams
        frames = []
        for _, row in allu_year.iterrows():
            portion_interest = sum([row['A_interest'], row['B_interest'], row['C_interest']]) / 100000000
            portion_other = (0.95 - portion_interest) * sum([row['A_balance'], row['B_balance'], row['C_balance'], row['A_principal'], row['B_principal'], row['C_principal']]) / 100000000

            frame = go.Frame(
                data=[
                    go.Sankey(
                        domain=dict(x=[0, 1], y=[0.95*portion_interest+0.05, 0.95*portion_interest+0.05 + portion_other]),
                        node=dict(
                            label=labels_principal
                        ),
                        link=dict(
                            source=[0, 0, 0, 4, 4, 4],
                            target=[1, 2, 3, 5, 6, 7],
                            value=[
                                row['A_balance'], row['B_balance'], row['C_balance'],
                                row['A_principal'], row['B_principal'], row['C_principal']
                            ],
                            color=link_colors[:6]
                        )
                    ),
                    go.Sankey(
                        domain=dict(x=[0, 1], y=[0, 0.95*portion_interest]),
                        node=dict(
                            label=labels_interest
                        ),
                        link=dict(
                            source=[0, 0, 0],
                            target=[1, 2, 3],
                            value=[
                                row['A_interest'], row['B_interest'], row['C_interest']
                            ],
                            color=link_colors[6:]
                        )
                    )
                ],
                name=str(int(row['period']))
            )
            frames.append(frame)

        # Add animation controls
        fig.update_layout(
            title_text="Animated Waterfall Model Sankey Diagram with Split Interest & Principal",
            font_size=12,
            height=1200,  # Adjust height for better visualization
            width=800,  # Adjust width as needed
            updatemenus=[{
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 2000, "redraw": True},
                                        "fromcurrent": True, "transition": {"duration": 300}}],
                        "label": "Play",
                        "method": "animate"
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                          "mode": "immediate",
                                          "transition": {"duration": 0}}],
                        "label": "Pause",
                        "method": "animate"
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.05,
                "xanchor": "right",
                "y": -0.03,
                "yanchor": "top"
            }],
            sliders=[{
                "steps": [
                    {
                        "args": [
                            [str(period)],
                            {"frame": {"duration": 300, "redraw": True},
                             "mode": "immediate", "transition": {"duration": 300}}
                        ],
                        "label": str(period),
                        "method": "animate"
                    }
                    for period in allu_year['period']
                ]
            }]
        )

        fig.frames = frames

        fig.show()


    def get_summary_stats(self, bonds_cashflows):
        """
        Calculates summary statistics for the bonds, including Weighted Average Life (WAL),
        total cash flow, and internal rate of return (IRR).

        Parameters
        ----------
        bonds_cashflows : pd.DataFrame
            A DataFrame containing the cash flow distribution and outstanding balances for each bond.

        Returns
        -------
        pd.DataFrame
            A DataFrame containing the WAL, total cash flow, and IRR for each bond.
        """

        # stat_dict = {'WAL': [], 'total_cashflow': [], 'irr': []}
        stat_dict = {'WAL': [], 'total_cashflow': []}
        for bond in self.bonds:
            # WAL
            year_principal_dict = {
                int((i/12+1)): sum(bonds_cashflows.iloc[i:i+12][f'{bond.class_name}_principal'].tolist()) for i in range(0, self.max_period, 12)
            } # dict of (year: year's cashflow)
            year, principals = np.array(list(year_principal_dict.keys())), np.array(list(year_principal_dict.values()))
            stat_dict['WAL'].append(principals/principals.sum() @ year)

            # total cashflow
            stat_dict['total_cashflow'].append(bonds_cashflows[f'{bond.class_name}_cashflow'].sum())

            # # irr; monthly irr, if want to convert (*12) to get annual irr
            # stat_dict['irr'].append(
            #     npf.irr(
            #         [-bond.initial_balance] + bonds_cashflows[f'{bond.class_name}_cashflow'].tolist()
            #     )
            # )

        return pd.DataFrame(stat_dict)

# Main Function

In [ ]:
def main():
    """
    Main function to execute the mortgage loan and bond cash flow analysis.
    """
    # pd.set_option("display.max_rows", None) [you can uncomment if you want to show the full dataframe below]

    # Load loan data
    file_path = 'Loan_tape_data.xlsx' # Change to specific path; I use it because I put the data and .ipynb file in the same folder
    data_loader = DataLoader(file_path)
    tape = data_loader.load_data()

    # Constants
    MAX_PERIOD = max(tape.Remaining_Amortization)

    # Initialize mortgage loans
    mortgage_loans = [MortgageLoan(*row.tolist()) for _, row in tape.iterrows()]

    # Initialize mortgage pool
    mp = MortgagePool(mortgage_loans=mortgage_loans)

    # Calculate mortgage cash flow
    individual_mortgage_cashflows, aggregate_mortgage_cashflow = mp.aggregate_cashflow()
    print("------------------------------------------------------------")
    print("Aggregated cashflow from all mortgages on a monthly basis...")
    display(aggregate_mortgage_cashflow)

    # Initialize bonds
    bonds_data = [['A', 70000000, 70000000], ['B', 20000000, 20000000], ['C', 10000000, 10000000]]
    bonds = [Bond(*bond) for bond in bonds_data]

    # Distribute cash flow from mortgage to bond
    wf = Waterfall(aggregate_mortgage_cashflow, bonds)
    bonds_cashflows = wf.distribute_cashflow()
    print("----------------------------------------")
    print("Waterfall model cashflow distribution...")
    display(bonds_cashflows)

    # Draw waterfall cashflow distribution animated diagram
    print("-------------------------------------")
    print("Drawing Cashflow animated diagram...")
    wf.animate_cashflow_diagram(bonds_cashflows)

    # Print summary statistics
    print("Summary statistics:")
    print(wf.get_summary_stats(bonds_cashflows))

# Execution

### Run instructions:

After executing the code below, there're 4 things:

1. Aggregated mortgage cashflow dataframe
2. Waterfall model cashflow distribution dataframe
3. An Animated Waterfall Model Sankey Diagram
4. Summary statistics

Instructions for using the Animated Waterfall Model Sankey Diagram:
(I recommend zooming out your screen to see the full diagram for a better view)

1. Click "play" or "pause" to start or stop the animation showing the dynamic changes in the waterfall model's distribution.
2. Use the slider to view the specific distribution for each year. Hover over elements to see detailed distribution information.


[Pay Attention]:

Make sure the data file **"Loan_tape_data.xlsx" is under the same directory** of the code.

The initial preview of the Animated Waterfall Model Sankey Diagram may not be accurate. Please ignore it. When playing the animation or drag the slider, everything will display normally. This issue may be addressed in a future update if possible.

In [ ]:
if __name__ == "__main__":
    main()

------------------------------------------------------------
Aggregated cashflow from all mortgages on a monthly basis...


,period,starting_balance,interest,principal,cashflow,ending_balance,effective_coupon
0,1,1.000000e+08,506458.333333,199790.348596,706248.681930,9.980021e+07,6.077500
1,2,9.980021e+07,505479.085739,200769.596191,706248.681930,9.959944e+07,6.077892
2,3,9.959944e+07,504494.984048,201753.697882,706248.681930,9.939769e+07,6.078287
3,4,9.939769e+07,503506.003922,202742.678007,706248.681930,9.919494e+07,6.078685
4,5,9.919494e+07,502512.120900,203736.561030,706248.681930,9.899121e+07,6.079086
...,...,...,...,...,...,...,...
355,356,1.794901e+06,9470.476048,355212.157259,364682.633307,1.439689e+06,6.331588
356,357,1.439689e+06,7596.531375,357086.101932,364682.633307,1.082602e+06,6.331812
357,358,1.082602e+06,5712.565827,358970.067480,364682.633307,7.236324e+05,6.332037
358,359,7.236324e+05,3818.525160,360864.108147,364682.633307,3.627683e+05,6.332262


----------------------------------------
Waterfall model cashflow distribution...


,period,A_balance,A_interest,A_principal,A_cashflow,B_balance,B_interest,B_principal,B_cashflow,C_balance,C_interest,C_principal,C_cashflow
0,1,6.980021e+07,354520.833333,199790.348596,554311.181930,20000000.0,101291.666667,0.0,101291.666667,1.000000e+07,50645.833333,0.000000,50645.833333
1,2,6.959944e+07,353531.783973,200769.596191,554301.380164,20000000.0,101298.201177,0.0,101298.201177,1.000000e+07,50649.100588,0.000000,50649.100588
2,3,6.939769e+07,352537.809258,201753.697882,554291.507140,20000000.0,101304.783193,0.0,101304.783193,1.000000e+07,50652.391597,0.000000,50652.391597
3,4,6.919494e+07,351538.884050,202742.678007,554281.562058,20000000.0,101311.413248,0.0,101311.413248,1.000000e+07,50655.706624,0.000000,50655.706624
4,5,6.899121e+07,350534.983076,203736.561030,554271.544106,20000000.0,101318.091883,0.0,101318.091883,1.000000e+07,50659.045941,0.000000,50659.045941
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.439689e+06,9470.476048,355212.157259,364682.633307
356,357,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.082602e+06,7596.531375,357086.101932,364682.633307
357,358,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,7.236324e+05,5712.565827,358970.067480,364682.633307
358,359,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,3.627683e+05,3818.525160,360864.108147,364682.633307


-------------------------------------
Drawing Cashflow animated diagram...


Summary statistics:
         WAL  total_cashflow
0  10.782864    1.143612e+08
1  24.999606    5.045311e+07
2  29.243057    2.795335e+07
